# Cài đặt thư viện

In [ ]:
!pip install paddleocr

In [ ]:
# Cài PaddlePaddle GPU 3.x (build cho CUDA 12.0, chạy tốt trên driver 12.5 của bạn)
!python -m pip install paddlepaddle-gpu==3.1.1 -i https://www.paddlepaddle.org.cn/packages/stable/cu126/

In [ ]:
!nvcc --version     # kiểm tra version CUDA
!python --version   # kiểm tra version Python

In [ ]:
import paddle
import paddleocr
print(paddle.__version__)
print(paddleocr.__version__)

In [ ]:
!nvcc --version

In [ ]:
import paddle
print(paddle.__version__)
print("CUDA build:", paddle.device.is_compiled_with_cuda())
print("GPU count:", paddle.device.cuda.device_count())
paddle.utils.run_check()

# Khởi tạo mô hình PaddleOCR

In [ ]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(
    device='gpu',
    lang='en',
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False)

In [ ]:
import time
start = time.time()
results = ocr.predict(
    input="/kaggle/input/sample-img/test.jpg")
end = time.time()
print(end - start)

In [ ]:
print(results)

In [ ]:
import numpy as np

# Nếu rec_boxes là polygon thì convert về [xmin,ymin,xmax,ymax]
boxes = result['rec_boxes']
texts = result['rec_texts']

if boxes.ndim == 3:  # [N,4,2]
    new_boxes = []
    for b in boxes:
        x_min, y_min = b[:,0].min(), b[:,1].min()
        x_max, y_max = b[:,0].max(), b[:,1].max()
        new_boxes.append([x_min, y_min, x_max, y_max])
    boxes = np.array(new_boxes)

def box_height(box):
    return box[3] - box[1]

def group_boxes_by_line(boxes, texts, y_threshold=20, size_ratio_threshold=1.8):
    idxs = np.argsort(boxes[:,1])  
    boxes, texts = boxes[idxs], np.array(texts)[idxs]

    lines = []
    current_line = [boxes[0]]
    current_texts = [texts[0]]

    for b, t in zip(boxes[1:], texts[1:]):
        prev = current_line[-1]

        same_line = abs(b[1] - prev[1]) < y_threshold
        size_ok = (max(box_height(b), box_height(prev)) / 
                   max(1, min(box_height(b), box_height(prev)))) < size_ratio_threshold

        if same_line and size_ok:
            current_line.append(b)
            current_texts.append(t)
        else:
            # kết thúc line hiện tại
            x_min = min(bb[0] for bb in current_line)
            y_min = min(bb[1] for bb in current_line)
            x_max = max(bb[2] for bb in current_line)
            y_max = max(bb[3] for bb in current_line)
            lines.append(((x_min, y_min, x_max, y_max), " ".join(current_texts)))

            # bắt đầu line mới
            current_line = [b]
            current_texts = [t]

    # Thêm line cuối
    if current_line:
        x_min = min(bb[0] for bb in current_line)
        y_min = min(bb[1] for bb in current_line)
        x_max = max(bb[2] for bb in current_line)
        y_max = max(bb[3] for bb in current_line)
        lines.append(((x_min, y_min, x_max, y_max), " ".join(current_texts)))

    return lines

# Thực thi
lines = group_boxes_by_line(boxes, texts, y_threshold=25, size_ratio_threshold=1.5)

# In kết quả
for (x_min, y_min, x_max, y_max), line_text in lines:
    print(f"Line box: ({x_min},{y_min},{x_max},{y_max}) -> {line_text}")

In [ ]:
import cv2
import os

# Load ảnh gốc
image_path = result['input_path']
img = cv2.imread(image_path)

# Folder lưu crop
out_dir = "line_crops"
os.makedirs(out_dir, exist_ok=True)

# Duyệt qua từng line box sau khi gộp
for i, (box, text) in enumerate(lines):
    x_min, y_min, x_max, y_max = map(int, box)

    # Crop theo bounding box
    crop = img[y_min:y_max, x_min:x_max]

    # Đặt tên file: line_1.jpg, line_2.jpg ...
    out_path = os.path.join(out_dir, f"line_{i+1}.jpg")
    cv2.imwrite(out_path, crop)

    print(f"✅ Saved {out_path} -> Text: {text}")

# Plot Kết quả bounding box và detect chữ

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Giả sử result là dict anh lấy từ ocr.predict()
result = results[0]   # vì results là list, mỗi ảnh 1 dict
image_path = result['input_path']

# Load ảnh gốc
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Lặp qua từng box + text
for poly, text, score in zip(result['rec_polys'], result['rec_texts'], result['rec_scores']):
    pts = np.array(poly, np.int32).reshape((-1, 1, 2))
    
    # Vẽ bounding box (đa giác)
    cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

    # Vẽ text OCR trên góc trái box
    x, y = pts[0][0]
    cv2.putText(img, f"{text} ({score:.2f})", (x, y-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

# Hiển thị kết quả
plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.axis("off")
plt.show()

# Xoá vùng không chứa chữ

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

# Kết quả OCR (dict cho 1 ảnh)
result = results[0]
image_path = result['input_path']

# Load ảnh gốc
img = cv2.imread(image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# --------- (1) Tạo ảnh chỉ giữ lại vùng chữ ---------
mask = np.zeros_like(img)

for poly in result['rec_polys']:
    pts = np.array(poly, np.int32).reshape((-1, 1, 2))
    cv2.fillPoly(mask, [pts], (255, 255, 255))

# Ảnh cuối cùng chỉ giữ chữ
only_text = cv2.bitwise_and(img, mask)

# Lưu ảnh này
cv2.imwrite("only_text.jpg", cv2.cvtColor(only_text, cv2.COLOR_RGB2BGR))
print("✅ Đã lưu ảnh only_text.jpg")

# --------- (2) Crop từng text box và lưu riêng ---------
os.makedirs("crops", exist_ok=True)  # tạo thư mục crops

for idx, poly in enumerate(result['rec_polys']):
    pts = np.array(poly, dtype=np.int32)  # ép kiểu
    if pts.shape[0] == 0:   # nếu poly rỗng thì bỏ qua
        continue

    x, y, w, h = cv2.boundingRect(pts)
    crop = img[y:y+h, x:x+w]

    out_path = f"crops/text_{idx+1}.jpg"
    cv2.imwrite(out_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))

print("✅ Đã lưu tất cả vùng chữ vào thư mục crops/")

In [ ]:
print("lhl")

In [ ]:
! pip install --quiet vietocr

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg

In [ ]:
config = Cfg.load_config_from_name('vgg_transformer')


In [ ]:
detector = Predictor(config)

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# Thư mục chứa ảnh crop
crop_dir = "/kaggle/working/line_crops"

# Duyệt toàn bộ file trong thư mục
for fname in os.listdir(crop_dir):
    if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(crop_dir, fname)
        
        # Load ảnh
        img = Image.open(img_path)
        plt.imshow(img)
        plt.title(fname)
        plt.axis("off")
        plt.show()
        
        # Predict bằng detector
        s = detector.predict(img)
        print(f"✅ {fname} -> {s}")